## 1️⃣ Kiểm Tra GPU

In [ ]:
# Check GPU availability
!nvidia-smi

import torch
print(f"\n✅ GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"📊 GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"💾 VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

## 2️⃣ Cài Đặt XTTS-v2

In [ ]:
# Install TTS library (XTTS-v2)
!pip install -q TTS
print("✅ Installation complete!")

## 3️⃣ Load Model XTTS-v2

In [ ]:
from TTS.api import TTS
import torch

# Load XTTS-v2 model
print("⏳ Loading XTTS-v2 model... (có thể mất 1-2 phút)")

device = "cuda" if torch.cuda.is_available() else "cpu"
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)

print(f"✅ Model loaded on {device}!")

## 4️⃣ Upload File MP3 Giọng Nói Reference

**Yêu cầu:**
- Độ dài: Tối thiểu 6 giây (khuyến nghị 10-30 giây)
- Chất lượng: Rõ ràng, ít noise
- Ngôn ngữ: Bất kỳ (tiếng Anh tốt nhất)

In [ ]:
from google.colab import files
import os

# Upload reference audio
print("📤 Chọn file MP3 giọng nói cần clone:")
uploaded = files.upload()

# Get first uploaded file
speaker_wav = list(uploaded.keys())[0]
print(f"\n✅ Uploaded: {speaker_wav}")

# Play uploaded audio to verify
from IPython.display import Audio, display
print("\n🔊 Preview giọng nói reference:")
display(Audio(speaker_wav))

## 5️⃣ Voice Cloning - Tạo Giọng Nói

Nhập text tiếng Anh và tạo giọng nói với voice đã clone

In [ ]:
# Input text
text = input("📝 Nhập text tiếng Anh (hoặc để trống để dùng demo): ")

if not text.strip():
    text = "Hello! This is a demo of voice cloning technology. The quick brown fox jumps over the lazy dog."
    print(f"Using demo text: {text}")

# Generate speech
print("\n⏳ Generating speech...")
output_path = "output.wav"

tts.tts_to_file(
    text=text,
    speaker_wav=speaker_wav,
    language="en",
    file_path=output_path
)

print(f"✅ Generated: {output_path}")

# Play output
print("\n🔊 Kết quả:")
display(Audio(output_path))

## 6️⃣ Download Kết Quả

In [ ]:
# Download generated audio
from google.colab import files

files.download(output_path)
print("📥 File đã được download!")

## 7️⃣ Batch Processing - Tạo Nhiều File

Nếu bạn muốn tạo nhiều đoạn audio cùng lúc

In [ ]:
# List of texts to generate
texts = [
    "Welcome to the English learning system.",
    "Practice makes perfect.",
    "The weather is beautiful today.",
    "Would you like some coffee?"
]

# Generate all
import os
os.makedirs("outputs", exist_ok=True)

print("⏳ Generating batch...\n")
for i, text in enumerate(texts, 1):
    output = f"outputs/output_{i}.wav"
    print(f"[{i}/{len(texts)}] {text[:50]}...")
    
    tts.tts_to_file(
        text=text,
        speaker_wav=speaker_wav,
        language="en",
        file_path=output
    )

print("\n✅ Batch generation complete!")

# Play all outputs
for i in range(1, len(texts) + 1):
    print(f"\n🔊 Output {i}:")
    display(Audio(f"outputs/output_{i}.wav"))

## 8️⃣ Tạo Giọng Đọc PDF/Text File

Upload file text hoặc PDF để chuyển thành audio

In [ ]:
# Install PDF reader
!pip install -q PyPDF2

from google.colab import files
import PyPDF2

# Upload file
print("📤 Upload file .txt hoặc .pdf:")
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# Read content
if filename.endswith('.pdf'):
    with open(filename, 'rb') as f:
        pdf = PyPDF2.PdfReader(f)
        text = ""
        for page in pdf.pages:
            text += page.extract_text()
else:
    with open(filename, 'r', encoding='utf-8') as f:
        text = f.read()

print(f"\n📄 Extracted {len(text)} characters")
print(f"Preview: {text[:200]}...\n")

# Split into chunks (XTTS works best with shorter texts)
chunk_size = 500
chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

print(f"📊 Split into {len(chunks)} chunks")

# Generate audio for each chunk
os.makedirs("pdf_outputs", exist_ok=True)

for i, chunk in enumerate(chunks, 1):
    if not chunk.strip():
        continue
    
    print(f"[{i}/{len(chunks)}] Generating...")
    output = f"pdf_outputs/chunk_{i:03d}.wav"
    
    tts.tts_to_file(
        text=chunk,
        speaker_wav=speaker_wav,
        language="en",
        file_path=output
    )

print("\n✅ PDF conversion complete!")
print(f"📁 Files saved in pdf_outputs/ folder")

## 9️⃣ Merge All Audio Files (Optional)

Gộp tất cả các chunks thành 1 file duy nhất

In [ ]:
!pip install -q pydub

from pydub import AudioSegment
import glob

# Get all chunk files
chunks = sorted(glob.glob("pdf_outputs/chunk_*.wav"))

if chunks:
    print(f"⏳ Merging {len(chunks)} files...")
    
    combined = AudioSegment.empty()
    for chunk in chunks:
        audio = AudioSegment.from_wav(chunk)
        combined += audio
        combined += AudioSegment.silent(duration=500)  # 500ms pause between chunks
    
    # Export
    output_file = "final_audiobook.mp3"
    combined.export(output_file, format="mp3")
    
    print(f"\n✅ Merged into: {output_file}")
    print(f"⏱️ Duration: {len(combined)/1000:.1f} seconds")
    
    # Download
    files.download(output_file)
else:
    print("❌ No chunks found to merge")

---

## 📊 Tips & Best Practices

### Reference Audio Quality:
- ✅ Độ dài: 10-30 giây là tốt nhất
- ✅ Rõ ràng, ít noise, không có nhạc nền
- ✅ Một người nói duy nhất
- ✅ Âm lượng ổn định

### Performance:
- **GPU T4**: ~5-10 giây/câu
- **GPU V100**: ~2-5 giây/câu
- Colab Free có thể disconnect sau 12 giờ

### Limitations:
- Colab free không đảm bảo luôn có GPU
- Không lưu file sau khi disconnect
- Cần re-upload reference audio mỗi session

### Next Steps:
- Upgrade Colab Pro ($10/tháng) cho GPU ổn định hơn
- Thử các model khác: Fish Speech, GPT-SoVITS
- Deploy lên RunPod cho production use

---

**🎉 Chúc bạn voice cloning thành công!**